## **1. Setup and Dependencies**

In [ ]:
# Install required packages
!pip install -q PyPDF2 pdf2image pytesseract pillow pydub gTTS
!pip install -q google-generativeai python-pptx
!pip install -q moviepy
!pip install -q transformers torch

print("✓ All packages installed successfully!")

In [ ]:
# Import libraries
import os
import re
import json
import logging
from pathlib import Path
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass

# PDF and text processing
from PyPDF2 import PdfReader
from pdf2image import convert_from_path
import pytesseract

# AI and NLP
import google.generativeai as genai
import transformers

# Audio processing
from gtts import gTTS
from pydub import AudioSegment

# PowerPoint
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor

# Video processing
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✓ All imports successful!")

## **2. Configuration Management**

In [ ]:
@dataclass
class Config:
    """Configuration class for the Research Assistant"""
    
    # API Configuration
    gemini_api_key: str = ""
    
    # Output directories
    output_dir: Path = Path("output")
    audio_dir: Path = Path("output/audio")
    video_dir: Path = Path("output/video")
    ppt_dir: Path = Path("output/presentations")
    
    # Processing parameters
    max_text_length: int = 30000
    summary_style: str = "detailed"  # Options: brief, detailed, technical
    
    # Audio settings
    audio_language: str = "en"
    audio_slow: bool = False
    
    def __post_init__(self):
        """Create directories if they don't exist"""
        for directory in [self.output_dir, self.audio_dir, self.video_dir, self.ppt_dir]:
            directory.mkdir(parents=True, exist_ok=True)
    
    def configure_gemini(self):
        """Configure Gemini API"""
        if not self.gemini_api_key:
            raise ValueError("Gemini API key not set!")
        genai.configure(api_key=self.gemini_api_key)
        logger.info("✓ Gemini API configured")

# Initialize configuration
config = Config()

# Set your API key here
config.gemini_api_key = "YOUR_API_KEY_HERE"  # Replace with your actual API key
config.configure_gemini()

## **3. Text Extraction Module**

In [ ]:
class PDFExtractor:
    """Enhanced PDF text extraction with fallback mechanisms"""
    
    @staticmethod
    def extract_text(pdf_path: str) -> str:
        """Extract text from PDF with OCR fallback"""
        logger.info(f"Extracting text from: {pdf_path}")
        
        text = ""
        try:
            # Try standard extraction first
            reader = PdfReader(pdf_path)
            for i, page in enumerate(reader.pages):
                page_text = page.extract_text()
                if page_text.strip():
                    text += page_text + "\n"
            
            # If no text extracted, use OCR
            if len(text.strip()) < 100:
                logger.info("Standard extraction yielded little text, trying OCR...")
                text = PDFExtractor._ocr_extract(pdf_path)
            
            logger.info(f"✓ Extracted {len(text)} characters")
            return text
            
        except Exception as e:
            logger.error(f"Error in text extraction: {e}")
            logger.info("Attempting OCR fallback...")
            return PDFExtractor._ocr_extract(pdf_path)
    
    @staticmethod
    def _ocr_extract(pdf_path: str) -> str:
        """Extract text using OCR"""
        try:
            images = convert_from_path(pdf_path)
            text = ""
            for i, img in enumerate(images):
                logger.info(f"OCR processing page {i+1}/{len(images)}")
                text += pytesseract.image_to_string(img) + "\n"
            return text
        except Exception as e:
            logger.error(f"OCR extraction failed: {e}")
            raise
    
    @staticmethod
    def preprocess_text(text: str) -> str:
        """Clean and preprocess extracted text"""
        # Remove page numbers and headers
        text = re.sub(r'Page \d+ of \d+', '', text)
        text = re.sub(r'\d+\s*\n', '', text)  # Remove standalone page numbers
        
        # Remove excessive whitespace
        text = re.sub(r'\n\s*\n', '\n\n', text)
        text = re.sub(r' +', ' ', text)
        
        # Remove special characters that might cause issues
        text = text.replace('\x00', '')
        
        return text.strip()

# Test the extractor
print("✓ PDF Extractor module loaded")

## **4. AI Processing Module**

In [ ]:
class GeminiProcessor:
    """Enhanced Gemini API processor with retry logic"""
    
    def __init__(self, model_name: str = 'gemini-pro'):
        self.model = genai.GenerativeModel(model_name)
        self.generation_config = genai.GenerationConfig(
            temperature=0.7,
            top_p=0.95,
            top_k=40,
            max_output_tokens=8192,
        )
    
    def generate_content(self, prompt: str, max_retries: int = 3) -> str:
        """Generate content with retry logic"""
        for attempt in range(max_retries):
            try:
                response = self.model.generate_content(
                    prompt,
                    generation_config=self.generation_config
                )
                return response.text.strip()
            except Exception as e:
                logger.warning(f"Attempt {attempt + 1} failed: {e}")
                if attempt == max_retries - 1:
                    raise
        return ""
    
    def summarize_research_paper(self, text: str, style: str = "detailed") -> str:
        """Generate research paper summary with improved prompt"""
        
        style_instructions = {
            "brief": "Provide a concise 200-300 word summary focusing only on the main contribution and key findings.",
            "detailed": "Provide a comprehensive 500-700 word summary covering all major aspects.",
            "technical": "Provide a detailed technical summary preserving important terminology and methodological details."
        }
        
        prompt = f"""
You are an expert research paper analyst. {style_instructions.get(style, style_instructions['detailed'])}

Structure your summary as a cohesive narrative covering:
1. Research Context & Problem Statement
2. Novel Contribution & Approach
3. Methodology & Experimental Design
4. Key Results & Analysis
5. Implications & Impact
6. Limitations & Future Work

Write in clear, flowing paragraphs. Avoid bullet points, asterisks, or formatting markers.
Make it accessible to someone with general scientific literacy but not necessarily expertise in the specific field.

Research Paper:
{text[:config.max_text_length]}
"""
        
        logger.info("Generating summary...")
        summary = self.generate_content(prompt)
        logger.info(f"✓ Summary generated ({len(summary)} characters)")
        return summary
    
    def simplify_with_analogies(self, text: str) -> str:
        """Simplify technical text using analogies"""
        
        prompt = f"""
You are an expert at explaining complex concepts through relatable analogies.

Transform the following technical summary into an accessible explanation using:
- Everyday analogies (compare to familiar objects, processes, or experiences)
- Simple language (8th-grade reading level)
- Concrete examples rather than abstract concepts
- Progressive explanation (build from simple to complex)

Maintain accuracy while making it engaging and memorable.

Technical Summary:
{text}

Simplified Explanation:
"""
        
        logger.info("Simplifying text with analogies...")
        simplified = self.generate_content(prompt)
        logger.info("✓ Text simplified")
        return simplified
    
    def generate_core_analogy(self, concept: str) -> str:
        """Generate a specific analogy for a core concept"""
        
        prompt = f"""
Create a clear, relatable analogy for this concept: {concept}

Format:
[Concept] is like [Familiar Thing] because [Explanation]

Make it memorable and accurate. Use one of these analogy types:
- Physical object comparison
- Process/workflow comparison  
- Social/organizational comparison
- Natural phenomenon comparison
"""
        
        return self.generate_content(prompt)

# Initialize processor
gemini = GeminiProcessor()
print("✓ Gemini Processor initialized")

## **5. Podcast Generation Module**

In [ ]:
class PodcastGenerator:
    """Enhanced podcast script and audio generation"""
    
    def __init__(self, gemini_processor: GeminiProcessor):
        self.gemini = gemini_processor
    
    def generate_script(self, summary: str, analogy: str, num_exchanges: int = 5) -> Dict[str, List[str]]:
        """Generate structured podcast script"""
        
        prompt = f"""
Create an engaging podcast dialogue between a Host and an Expert researcher.

Structure: {num_exchanges} question-answer exchanges

Requirements:
1. Host asks insightful questions that build understanding progressively
2. Expert explains using the provided analogy: {analogy}
3. Each exchange should:
   - Start conversational and engaging
   - Use the analogy to clarify technical concepts
   - Connect back to real-world applications
4. Include natural transitions between topics

Format each line EXACTLY as:
Host: [question]
Expert: [answer using analogy]

Research Summary:
{summary[:15000]}

Generate {num_exchanges} exchanges:
"""
        
        logger.info("Generating podcast script...")
        script_text = self.gemini.generate_content(prompt)
        
        # Parse script into structured format
        script = self._parse_script(script_text)
        
        # Validate script
        if not script['host'] or not script['expert']:
            logger.warning("Script parsing failed, using fallback")
            script = self._generate_fallback_script(summary, analogy)
        
        logger.info(f"✓ Script generated: {len(script['host'])} exchanges")
        return script
    
    def _parse_script(self, script_text: str) -> Dict[str, List[str]]:
        """Parse script text into structured format"""
        host_lines = []
        expert_lines = []
        
        for line in script_text.split('\n'):
            line = line.strip()
            if line.startswith("Host:"):
                host_lines.append(line.replace("Host:", "").strip())
            elif line.startswith("Expert:"):
                expert_lines.append(line.replace("Expert:", "").strip())
        
        return {'host': host_lines, 'expert': expert_lines}
    
    def _generate_fallback_script(self, summary: str, analogy: str) -> Dict[str, List[str]]:
        """Generate a basic fallback script"""
        return {
            'host': [
                "Welcome to our research podcast! Can you explain what this study is about?",
                "That's fascinating! What was the main challenge you were trying to solve?",
                "How did you approach this problem?",
                "What were the key findings?",
                "What does this mean for the future?"
            ],
            'expert': [
                f"Let me explain using this analogy: {analogy}. {summary[:200]}...",
                f"The main challenge relates to how {analogy.split()[0]} works...",
                "Our approach was systematic and data-driven...",
                "We discovered several important insights...",
                "This research opens up exciting new possibilities..."
            ]
        }
    
    def generate_audio(self, script: Dict[str, List[str]], output_path: str) -> str:
        """Generate podcast audio from script"""
        logger.info("Generating podcast audio...")
        
        audio_segments = []
        temp_files = []
        
        try:
            # Generate audio for each exchange
            for i, (host_line, expert_line) in enumerate(zip(script['host'], script['expert'])):
                # Host audio (slightly faster)
                host_file = f"temp_host_{i}.mp3"
                host_tts = gTTS(host_line, lang=config.audio_language, slow=False)
                host_tts.save(host_file)
                temp_files.append(host_file)
                
                # Expert audio (normal pace)
                expert_file = f"temp_expert_{i}.mp3"
                expert_tts = gTTS(expert_line, lang=config.audio_language, slow=config.audio_slow)
                expert_tts.save(expert_file)
                temp_files.append(expert_file)
                
                # Load and combine
                host_audio = AudioSegment.from_mp3(host_file)
                expert_audio = AudioSegment.from_mp3(expert_file)
                
                # Add pause between speakers
                audio_segments.append(host_audio)
                audio_segments.append(AudioSegment.silent(duration=800))  # 0.8 second pause
                audio_segments.append(expert_audio)
                audio_segments.append(AudioSegment.silent(duration=1200))  # 1.2 second pause
            
            # Combine all segments
            final_audio = sum(audio_segments)
            
            # Export
            final_audio.export(output_path, format="mp3")
            logger.info(f"✓ Podcast audio saved: {output_path}")
            
            return output_path
            
        finally:
            # Cleanup temp files
            for temp_file in temp_files:
                if os.path.exists(temp_file):
                    os.remove(temp_file)

# Initialize generator
podcast_gen = PodcastGenerator(gemini)
print("✓ Podcast Generator initialized")

## **6. PowerPoint Generation Module**

In [ ]:
# Theme definitions
THEMES = {
    "professional": {
        "title_font": "Calibri",
        "title_size": Pt(44),
        "title_color": RGBColor(31, 78, 121),
        "title_bold": True,
        "body_font": "Calibri",
        "body_size": Pt(18),
        "body_color": RGBColor(64, 64, 64),
        "background_color": RGBColor(255, 255, 255),
        "accent_color": RGBColor(68, 114, 196)
    },
    "modern": {
        "title_font": "Arial",
        "title_size": Pt(40),
        "title_color": RGBColor(47, 84, 150),
        "title_bold": True,
        "body_font": "Arial",
        "body_size": Pt(16),
        "body_color": RGBColor(89, 89, 89),
        "background_color": RGBColor(248, 248, 248),
        "accent_color": RGBColor(255, 192, 0)
    },
    "creative": {
        "title_font": "Georgia",
        "title_size": Pt(42),
        "title_color": RGBColor(192, 80, 77),
        "title_bold": True,
        "body_font": "Georgia",
        "body_size": Pt(18),
        "body_color": RGBColor(79, 79, 79),
        "background_color": RGBColor(252, 248, 227),
        "accent_color": RGBColor(142, 169, 219)
    }
}

class PowerPointGenerator:
    """Enhanced PowerPoint presentation generator"""
    
    def __init__(self, gemini_processor: GeminiProcessor):
        self.gemini = gemini_processor
    
    def generate_content(self, summary: str, num_slides: int = 8) -> str:
        """Generate structured slide content"""
        
        prompt = f"""
Create a {num_slides}-slide PowerPoint presentation from this research paper summary.

Required Structure:
Slide 1: Title Slide
- Title: [Concise research title]
- Subtitle: [Key focus area]
- Presented by: Deep Drillers

Slide 2: Research Context
- Problem statement
- Motivation
- Research gap

Slide 3: Objectives & Approach
- Primary objectives (3-4 points)
- Research questions
- Overall approach

Slide 4: Methodology
- Research design
- Data sources/collection
- Analysis methods

Slide 5: Key Findings
- Finding 1 (with metric if available)
- Finding 2 (with metric if available)
- Finding 3 (with metric if available)

Slide 6: Analysis & Discussion
- Interpretation of results
- Comparison with existing work
- Unexpected insights

Slide 7: Impact & Applications
- Theoretical contributions
- Practical applications
- Future research directions

Slide 8: Conclusion
- Main takeaways (3 points)
- Limitations
- Closing statement

Format Requirements:
- Start each slide with "Slide N: [Title]"
- Use bullet points (start with "- ")
- Keep bullets concise (max 15 words each)
- No asterisks or special formatting

Summary:
{summary}
"""
        
        logger.info("Generating presentation content...")
        content = self.gemini.generate_content(prompt)
        content = content.replace("**", "").replace("*", "")
        logger.info("✓ Presentation content generated")
        return content
    
    def create_presentation(self, content: str, theme_name: str, output_path: str) -> str:
        """Create PowerPoint presentation with theme"""
        logger.info(f"Creating presentation with '{theme_name}' theme...")
        
        # Parse content
        slides_data = self._parse_content(content)
        
        if not slides_data:
            raise ValueError("Failed to parse presentation content")
        
        # Get theme
        theme = THEMES.get(theme_name, THEMES["professional"])
        
        # Create presentation
        prs = Presentation()
        prs.slide_width = Inches(10)
        prs.slide_height = Inches(7.5)
        
        for i, slide_data in enumerate(slides_data):
            self._add_slide(prs, slide_data, theme)
            logger.info(f"  Added slide {i+1}/{len(slides_data)}")
        
        # Add footer to all slides
        self._add_footer(prs, theme)
        
        # Save
        prs.save(output_path)
        logger.info(f"✓ Presentation saved: {output_path}")
        return output_path
    
    def _parse_content(self, content: str) -> List[Dict[str, any]]:
        """Parse content into structured slide data"""
        slides = []
        current_slide = {"title": "", "bullets": []}
        
        for line in content.split("\n"):
            line = line.strip()
            
            if line.startswith("Slide") and ":" in line:
                if current_slide["title"]:
                    slides.append(current_slide)
                    current_slide = {"title": "", "bullets": []}
                
                # Extract title
                title_part = line.split(": ", 1)
                if len(title_part) > 1:
                    current_slide["title"] = title_part[1]
            
            elif line.startswith("-") and line[1:].strip():
                current_slide["bullets"].append(line[1:].strip())
        
        if current_slide["title"]:
            slides.append(current_slide)
        
        return slides
    
    def _add_slide(self, prs: Presentation, slide_data: Dict, theme: Dict):
        """Add a slide to the presentation"""
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        
        # Set background
        background = slide.background
        fill = background.fill
        fill.solid()
        fill.fore_color.rgb = theme["background_color"]
        
        # Add title
        title = slide.shapes.title
        title.text = slide_data["title"]
        title_frame = title.text_frame
        title_para = title_frame.paragraphs[0]
        title_para.font.name = theme["title_font"]
        title_para.font.size = theme["title_size"]
        title_para.font.color.rgb = theme["title_color"]
        title_para.font.bold = theme["title_bold"]
        
        # Add content
        if len(slide.shapes.placeholders) > 1:
            content_box = slide.shapes.placeholders[1]
            text_frame = content_box.text_frame
            text_frame.clear()
            
            for bullet in slide_data["bullets"]:
                p = text_frame.add_paragraph()
                p.text = bullet
                p.level = 0
                p.font.name = theme["body_font"]
                p.font.size = theme["body_size"]
                p.font.color.rgb = theme["body_color"]
    
    def _add_footer(self, prs: Presentation, theme: Dict):
        """Add footer to all slides"""
        for slide in prs.slides:
            footer = slide.shapes.add_textbox(
                Inches(0.5), Inches(7), Inches(9), Inches(0.3)
            )
            text_frame = footer.text_frame
            p = text_frame.paragraphs[0]
            p.text = "Generated by Research Assistant | Powered by Gemini AI"
            p.font.size = Pt(10)
            p.font.color.rgb = RGBColor(128, 128, 128)
            p.alignment = 1  # Center

# Initialize generator
ppt_gen = PowerPointGenerator(gemini)
print("✓ PowerPoint Generator initialized")

## **7. Video Processing Module**

In [ ]:
class VideoProcessor:
    """Enhanced video processing with audio overlay"""
    
    @staticmethod
    def process_video(video_path: str, audio_path: str, output_path: str) -> str:
        """Add audio overlay to video"""
        logger.info(f"Processing video: {video_path}")
        
        try:
            # Load video and audio
            video_clip = VideoFileClip(video_path)
            audio_clip = AudioFileClip(audio_path)
            
            # Adjust video duration to match audio
            audio_duration = audio_clip.duration
            
            if video_clip.duration < audio_duration:
                # Loop video if it's shorter than audio
                num_loops = int(audio_duration / video_clip.duration) + 1
                video_clip = video_clip.loop(n=num_loops)
            
            # Cut to audio duration
            video_clip = video_clip.subclip(0, audio_duration)
            
            # Mix audio if video has original audio
            if video_clip.audio is not None:
                # Lower original audio volume
                original_audio = video_clip.audio.volumex(0.2)
                final_audio = CompositeAudioClip([original_audio, audio_clip])
            else:
                final_audio = audio_clip
            
            # Set audio to video
            final_clip = video_clip.set_audio(final_audio)
            
            # Write output
            final_clip.write_videofile(
                output_path,
                codec="libx264",
                audio_codec="aac",
                temp_audiofile="temp_audio.m4a",
                remove_temp=True
            )
            
            # Cleanup
            video_clip.close()
            audio_clip.close()
            final_clip.close()
            
            logger.info(f"✓ Video processed: {output_path}")
            return output_path
            
        except Exception as e:
            logger.error(f"Video processing error: {e}")
            raise
    
    @staticmethod
    def batch_process_videos(video_paths: List[str], audio_path: str, output_dir: str) -> List[str]:
        """Process multiple videos with the same audio"""
        output_paths = []
        
        for i, video_path in enumerate(video_paths):
            output_path = os.path.join(output_dir, f"output_video_{i+1}.mp4")
            try:
                VideoProcessor.process_video(video_path, audio_path, output_path)
                output_paths.append(output_path)
            except Exception as e:
                logger.error(f"Failed to process {video_path}: {e}")
        
        return output_paths

print("✓ Video Processor initialized")

## **8. Main Processing Pipeline**

In [ ]:
class ResearchAssistant:
    """Main orchestration class for research paper processing"""
    
    def __init__(self, config: Config):
        self.config = config
        self.pdf_extractor = PDFExtractor()
        self.gemini = GeminiProcessor()
        self.podcast_gen = PodcastGenerator(self.gemini)
        self.ppt_gen = PowerPointGenerator(self.gemini)
    
    def process_paper(self, pdf_path: str, generate_all: bool = True) -> Dict[str, str]:
        """Complete processing pipeline for a research paper"""
        logger.info(f"\n{'='*60}")
        logger.info(f"Starting research paper processing")
        logger.info(f"{'='*60}\n")
        
        results = {}
        
        try:
            # 1. Extract text
            logger.info("[1/6] Extracting text from PDF...")
            raw_text = self.pdf_extractor.extract_text(pdf_path)
            clean_text = self.pdf_extractor.preprocess_text(raw_text)
            results['text'] = clean_text
            
            # 2. Generate summary
            logger.info("[2/6] Generating summary...")
            summary = self.gemini.summarize_research_paper(
                clean_text,
                style=self.config.summary_style
            )
            results['summary'] = summary
            
            # Save summary
            summary_path = self.config.output_dir / "summary.txt"
            with open(summary_path, 'w', encoding='utf-8') as f:
                f.write(summary)
            logger.info(f"  Summary saved: {summary_path}")
            
            # 3. Generate simplified version
            logger.info("[3/6] Creating simplified explanation...")
            simplified = self.gemini.simplify_with_analogies(summary)
            results['simplified'] = simplified
            
            # Save simplified version
            simplified_path = self.config.output_dir / "simplified.txt"
            with open(simplified_path, 'w', encoding='utf-8') as f:
                f.write(simplified)
            logger.info(f"  Simplified version saved: {simplified_path}")
            
            # 4. Generate core analogy
            logger.info("[4/6] Generating core analogy...")
            # Extract main concept from first paragraph
            first_para = summary.split('\n')[0]
            analogy = self.gemini.generate_core_analogy(first_para[:200])
            results['analogy'] = analogy
            logger.info(f"  Analogy: {analogy[:100]}...")
            
            if generate_all:
                # 5. Generate podcast
                logger.info("[5/6] Generating podcast...")
                script = self.podcast_gen.generate_script(summary, analogy)
                results['podcast_script'] = script
                
                # Save script
                script_path = self.config.audio_dir / "podcast_script.json"
                with open(script_path, 'w', encoding='utf-8') as f:
                    json.dump(script, f, indent=2)
                logger.info(f"  Script saved: {script_path}")
                
                # Generate audio
                audio_path = self.config.audio_dir / "podcast.mp3"
                self.podcast_gen.generate_audio(script, str(audio_path))
                results['podcast_audio'] = str(audio_path)
                
                # 6. Generate presentation
                logger.info("[6/6] Generating PowerPoint presentation...")
                ppt_content = self.ppt_gen.generate_content(summary)
                ppt_path = self.config.ppt_dir / "presentation.pptx"
                self.ppt_gen.create_presentation(
                    ppt_content,
                    "professional",
                    str(ppt_path)
                )
                results['presentation'] = str(ppt_path)
            
            logger.info(f"\n{'='*60}")
            logger.info("✓ Processing complete!")
            logger.info(f"{'='*60}\n")
            
            return results
            
        except Exception as e:
            logger.error(f"Processing failed: {e}")
            raise

# Initialize assistant
assistant = ResearchAssistant(config)
print("✓ Research Assistant initialized and ready!")

## **9. Usage Example**

In [ ]:
# Set your PDF path
PDF_PATH = "/content/your_paper.pdf"  # Change this to your PDF path

# Process the paper
results = assistant.process_paper(PDF_PATH, generate_all=True)

# Display results
print("\n" + "="*60)
print("PROCESSING RESULTS")
print("="*60)
print(f"\n📄 Summary Length: {len(results['summary'])} characters")
print(f"\n📝 Simplified Version Length: {len(results['simplified'])} characters")
print(f"\n🎯 Core Analogy: {results['analogy']}")

if 'podcast_audio' in results:
    print(f"\n🎙️ Podcast Audio: {results['podcast_audio']}")
if 'presentation' in results:
    print(f"\n📊 Presentation: {results['presentation']}")

print("\n" + "="*60)

## **10. Optional: Generate Video**

In [ ]:
# If you have video files and want to add the podcast audio
VIDEO_PATH = "/content/your_video.mp4"  # Change this
AUDIO_PATH = results.get('podcast_audio', '')

if AUDIO_PATH and os.path.exists(VIDEO_PATH):
    output_video = config.video_dir / "final_video.mp4"
    VideoProcessor.process_video(VIDEO_PATH, AUDIO_PATH, str(output_video))
    print(f"✓ Video with audio overlay: {output_video}")
else:
    print("⚠️ Video or audio not available")

## **11. Download Files (Colab Only)**

In [ ]:
# Download generated files in Google Colab
try:
    from google.colab import files
    
    print("Downloading generated files...")
    
    # Download podcast
    if 'podcast_audio' in results:
        files.download(results['podcast_audio'])
    
    # Download presentation
    if 'presentation' in results:
        files.download(results['presentation'])
    
    print("✓ Downloads complete!")
    
except ImportError:
    print("Not running in Colab - files saved to output directory")